In [14]:
import os
import datetime
import imageio
import skimage
import scipy # 

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
from IPython.display import Image

from tensorflow.python.keras.callbacks import TensorBoard
import smtplib

tf.logging.set_verbosity(tf.logging.ERROR)

In [15]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('%s/%s/*' % (self.dataset_name, data_type))
        
        batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = self.imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, self.img_res)
            img_B = scipy.misc.imresize(img_B, self.img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B


    def imread(self, path):
        return imageio.imread(path).astype(np.float)

In [16]:
class Pix2Pix():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'dataset'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = tf.keras.optimizers.Adam(0.00002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generator
        #-------------------------

        # Build the generator
        self.generator = self.build_generator()

        # Input images and their conditioning images
        img_A = tf.keras.layers.Input(shape=self.img_shape)
        img_B = tf.keras.layers.Input(shape=self.img_shape)

        # By conditioning on B generate a fake version of A
        fake_A = self.generator(img_B)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # Discriminators determines validity of translated images / condition pairs
        valid = self.discriminator([fake_A, img_B])
        
        optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

        self.combined = tf.keras.models.Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
        self.combined.compile(loss=['mse', 'mae'],
                              loss_weights=[1, 100],
                              optimizer=optimizer)
        
        #-------------------------
        #     TensorBoard Log
        #-------------------------
        
        log_path = 'logs'
        self.callback = TensorBoard(log_path)
        self.callback.set_model(self.combined)
        self.train_names = ['train_loss', 'train_mae']
        self.val_names = ['val_loss', 'val_mae']
    
    def write_log(self, callback, names, logs, batch_no):
        for name, value in zip(names, logs):
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value
            summary_value.tag = name
            callback.writer.add_summary(summary, batch_no)
            callback.writer.flush()

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = tf.keras.layers.Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
            if bn:
                d = tf.keras.layers.BatchNormalization(momentum=0.8)(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = tf.keras.layers.UpSampling2D(size=2)(layer_input)
            u = tf.keras.layers.Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = tf.keras.layers.Dropout(dropout_rate)(u)
            u = tf.keras.layers.BatchNormalization(momentum=0.8)(u)
            u = tf.keras.layers.Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = tf.keras.layers.Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, bn=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)
        print(d7.shape)
        print(d6.shape)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = tf.keras.layers.UpSampling2D(size=2)(u6)
        output_img = tf.keras.layers.Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return tf.keras.models.Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, bn=True):
            """Discriminator layer"""
            d = tf.keras.layers.Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = tf.keras.layers.LeakyReLU(alpha=0.2)(d)
            if bn:
                d = tf.keras.layers.BatchNormalization(momentum=0.8)(d)
            return d

        img_A = tf.keras.layers.Input(shape=self.img_shape)
        img_B = tf.keras.layers.Input(shape=self.img_shape)

        # Concatenate image and conditioning image by channels to produce input
        combined_imgs = tf.keras.layers.Concatenate(axis=-1)([img_A, img_B])

        d1 = d_layer(combined_imgs, self.df, bn=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)
        #print(d4.shape)

        validity = tf.keras.layers.Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return tf.keras.models.Model([img_A, img_B], validity)

    def train(self, epochs, batch_size=1, sample_interval=50):
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Condition on B and generate a translated version
                fake_A = self.generator.predict(imgs_B)
                

                # Train the discriminators (original images = real / generated = Fake)
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, epochs,
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0]/batch_size,
                                                                        elapsed_time))
                
                self.write_log(self.callback, self.val_names, g_loss, batch_i)

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)

    def sample_images(self, epoch, batch_i):
        os.makedirs('./images/%s' % self.dataset_name, exist_ok=True)
        r, c = 3, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=3, is_testing=True)
        fake_A = self.generator.predict(imgs_B)

        gen_imgs = np.concatenate([imgs_B, fake_A, imgs_A])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Condition', 'Generated', 'Original']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[i])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

In [17]:
gan = Pix2Pix()
gan.generator.summary()

(?, 1, 1, 512)
(?, 2, 2, 512)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 64, 64, 64)   3136        input_20[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_59 (LeakyReLU)      (None, 64, 64, 64)   0           conv2d_76[0][0]                  
__________________________________________________________________________________________________
conv2d_77 (Conv2D)              (None, 32, 32, 128)  131200      leaky_re_lu_59[0][0]             
_______________________________________________________________________________

In [23]:
# Increase batch_size gradually
# Start with batch of size 1
# then 10 after like 500 epochs
# then 100 after 500 more epochs
# also need more data
# and need to split into test and validation set
gan.train(epochs=10000, batch_size=70, sample_interval=32)

D:\Users\amm65\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
D:\Users\amm65\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


[Epoch 0/10000] [Batch 0/36] [D loss: 0.060739, acc:  99%] [G loss: 8.163401] time: 0:00:02.337290


D:\Users\amm65\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
D:\Users\amm65\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


[Epoch 0/10000] [Batch 1/36] [D loss: 0.041331, acc:  99%] [G loss: 7.342183] time: 0:00:05.151136
[Epoch 0/10000] [Batch 2/36] [D loss: 0.058731, acc:  99%] [G loss: 6.190771] time: 0:00:08.361073
[Epoch 0/10000] [Batch 3/36] [D loss: 0.085033, acc:  92%] [G loss: 7.058984] time: 0:00:11.011579
[Epoch 0/10000] [Batch 4/36] [D loss: 0.055301, acc:  96%] [G loss: 6.244478] time: 0:00:13.460945
[Epoch 0/10000] [Batch 5/36] [D loss: 0.048944, acc:  99%] [G loss: 6.487414] time: 0:00:16.004885
[Epoch 0/10000] [Batch 6/36] [D loss: 0.040115, acc:  99%] [G loss: 6.698098] time: 0:00:18.488662
[Epoch 0/10000] [Batch 7/36] [D loss: 0.035481, acc:  99%] [G loss: 7.343241] time: 0:00:20.947166
[Epoch 0/10000] [Batch 8/36] [D loss: 0.048071, acc:  99%] [G loss: 6.089492] time: 0:00:23.408010
[Epoch 0/10000] [Batch 9/36] [D loss: 0.050750, acc:  99%] [G loss: 6.378377] time: 0:00:25.907959
[Epoch 0/10000] [Batch 10/36] [D loss: 0.033004, acc:  99%] [G loss: 7.010770] time: 0:00:28.355035
[Epoch 0/

[Epoch 2/10000] [Batch 14/36] [D loss: 0.035042, acc:  99%] [G loss: 6.253051] time: 0:03:33.848896
[Epoch 2/10000] [Batch 15/36] [D loss: 0.034731, acc:  99%] [G loss: 6.540001] time: 0:03:36.329642
[Epoch 2/10000] [Batch 16/36] [D loss: 0.111741, acc:  91%] [G loss: 6.654017] time: 0:03:38.835039
[Epoch 2/10000] [Batch 17/36] [D loss: 0.046341, acc:  99%] [G loss: 6.297898] time: 0:03:41.303443
[Epoch 2/10000] [Batch 18/36] [D loss: 0.035787, acc:  99%] [G loss: 6.576979] time: 0:03:43.822834
[Epoch 2/10000] [Batch 19/36] [D loss: 0.020724, acc:  99%] [G loss: 7.497158] time: 0:03:46.291100
[Epoch 2/10000] [Batch 20/36] [D loss: 0.051787, acc:  98%] [G loss: 5.943864] time: 0:03:48.694744
[Epoch 2/10000] [Batch 21/36] [D loss: 0.066568, acc:  97%] [G loss: 6.255686] time: 0:03:50.986039
[Epoch 2/10000] [Batch 22/36] [D loss: 0.028330, acc:  99%] [G loss: 6.704976] time: 0:03:53.277009
[Epoch 2/10000] [Batch 23/36] [D loss: 0.037506, acc:  99%] [G loss: 6.644901] time: 0:03:55.561116


...

[Epoch 28/10000] [Batch 9/36] [D loss: 0.067088, acc:  97%] [G loss: 5.629587] time: 0:41:10.435915
[Epoch 28/10000] [Batch 10/36] [D loss: 0.026518, acc:  99%] [G loss: 6.062657] time: 0:41:12.801733
[Epoch 28/10000] [Batch 11/36] [D loss: 0.061139, acc:  98%] [G loss: 6.087672] time: 0:41:15.209801
[Epoch 28/10000] [Batch 12/36] [D loss: 0.047238, acc:  99%] [G loss: 5.368460] time: 0:41:17.661255
[Epoch 28/10000] [Batch 13/36] [D loss: 0.023676, acc:  99%] [G loss: 7.298233] time: 0:41:20.095563
[Epoch 28/10000] [Batch 14/36] [D loss: 0.027306, acc:  99%] [G loss: 5.552992] time: 0:41:22.554069
[Epoch 28/10000] [Batch 15/36] [D loss: 0.031042, acc:  99%] [G loss: 5.787525] time: 0:41:24.985495
[Epoch 28/10000] [Batch 16/36] [D loss: 0.045126, acc:  99%] [G loss: 6.214814] time: 0:41:27.413700
[Epoch 28/10000] [Batch 17/36] [D loss: 0.030363, acc:  99%] [G loss: 5.659244] time: 0:41:29.828221
[Epoch 28/10000] [Batch 18/36] [D loss: 0.046275, acc:  99%] [G loss: 6.160319] time: 0:41:3

[Epoch 30/10000] [Batch 21/36] [D loss: 0.061879, acc:  97%] [G loss: 5.091615] time: 0:44:28.898031
[Epoch 30/10000] [Batch 22/36] [D loss: 0.028585, acc:  99%] [G loss: 6.264400] time: 0:44:31.281553
[Epoch 30/10000] [Batch 23/36] [D loss: 0.034030, acc:  99%] [G loss: 5.761091] time: 0:44:33.660480
[Epoch 30/10000] [Batch 24/36] [D loss: 0.060558, acc:  98%] [G loss: 6.385343] time: 0:44:36.003589
[Epoch 30/10000] [Batch 25/36] [D loss: 0.046757, acc: 100%] [G loss: 6.304284] time: 0:44:38.339585
[Epoch 30/10000] [Batch 26/36] [D loss: 0.088098, acc:  94%] [G loss: 5.320852] time: 0:44:40.730683
[Epoch 30/10000] [Batch 27/36] [D loss: 0.076618, acc:  97%] [G loss: 5.474159] time: 0:44:43.003368
[Epoch 30/10000] [Batch 28/36] [D loss: 0.057399, acc:  97%] [G loss: 6.030283] time: 0:44:45.381115
[Epoch 30/10000] [Batch 29/36] [D loss: 0.020454, acc:  99%] [G loss: 5.554451] time: 0:44:47.749284
[Epoch 30/10000] [Batch 30/36] [D loss: 0.022414, acc:  99%] [G loss: 5.589322] time: 0:44:

[Epoch 32/10000] [Batch 33/36] [D loss: 0.073788, acc:  97%] [G loss: 5.505059] time: 0:47:52.427085
[Epoch 32/10000] [Batch 34/36] [D loss: 0.019674, acc:  99%] [G loss: 6.594120] time: 0:47:54.925378
[Epoch 33/10000] [Batch 0/36] [D loss: 0.052466, acc: 100%] [G loss: 5.767337] time: 0:47:57.373803
[Epoch 33/10000] [Batch 1/36] [D loss: 0.025053, acc:  99%] [G loss: 6.350683] time: 0:48:00.089973
[Epoch 33/10000] [Batch 2/36] [D loss: 0.022446, acc:  99%] [G loss: 4.857995] time: 0:48:02.514117
[Epoch 33/10000] [Batch 3/36] [D loss: 0.040272, acc:  99%] [G loss: 5.996594] time: 0:48:05.016523
[Epoch 33/10000] [Batch 4/36] [D loss: 0.022239, acc:  99%] [G loss: 5.524319] time: 0:48:07.425837
[Epoch 33/10000] [Batch 5/36] [D loss: 0.047262, acc:  99%] [G loss: 5.707425] time: 0:48:09.806366
[Epoch 33/10000] [Batch 6/36] [D loss: 0.067002, acc:  94%] [G loss: 5.841396] time: 0:48:12.373076
[Epoch 33/10000] [Batch 7/36] [D loss: 0.028735, acc:  99%] [G loss: 5.905837] time: 0:48:14.73522

[Epoch 35/10000] [Batch 10/36] [D loss: 0.029650, acc:  99%] [G loss: 6.163313] time: 0:51:17.001785
[Epoch 35/10000] [Batch 11/36] [D loss: 0.041019, acc:  99%] [G loss: 5.851412] time: 0:51:19.678005
[Epoch 35/10000] [Batch 12/36] [D loss: 0.049354, acc:  98%] [G loss: 5.662390] time: 0:51:22.329209
[Epoch 35/10000] [Batch 13/36] [D loss: 0.021973, acc:  99%] [G loss: 7.003219] time: 0:51:25.105841
[Epoch 35/10000] [Batch 14/36] [D loss: 0.046236, acc:  99%] [G loss: 5.809279] time: 0:51:27.581015
[Epoch 35/10000] [Batch 15/36] [D loss: 0.031232, acc:  99%] [G loss: 5.714329] time: 0:51:30.007020
[Epoch 35/10000] [Batch 16/36] [D loss: 0.060219, acc:  98%] [G loss: 5.795777] time: 0:51:32.379984
[Epoch 35/10000] [Batch 17/36] [D loss: 0.042764, acc:  99%] [G loss: 5.575788] time: 0:51:34.771940
[Epoch 35/10000] [Batch 18/36] [D loss: 0.037572, acc:  99%] [G loss: 5.742100] time: 0:51:37.094624
[Epoch 35/10000] [Batch 19/36] [D loss: 0.033866, acc:  99%] [G loss: 6.652607] time: 0:51:

KeyboardInterrupt: 

In [22]:
img = np.array([imageio.imread("testresized1.jpg").astype(np.float32)])/127.5 - 1.
print(img)
nu = gan.generator.predict(img)
imageio.imwrite("generate.jpg",nu[0])

FileNotFoundError: No such file: 'D:\Users\amm65\Desktop\color palette project\testresized1.jpg'